In [1]:
import pandas as pd
import glob
import os
import re
import datetime

#폴더 내 csv 파일 찾기
folder = input("데이터가 있는 폴더:")
csv_files = glob.glob(os.path.join(folder, "*.csv"))
for file in csv_files:
    print(f"{os.path.basename(file)}")
print(f"총{len(csv_files)}개의 csv 파일을 찾았습니다\n")

# 파일명을 통해 최근 순으로 파일 정리. 
most_recent_file = None
max_year=0
year_pattern = re.compile(r'\d{4}') #숫자 4개가 연속으로 이어질 때 파싱
files_with_year=[]
for file in csv_files:
    file_name = os.path.basename(file)
    match = year_pattern.search(file_name)
    if match:
        year= int(match.group())
        files_with_year.append((year, file))
files_with_year.sort(key=lambda x:x[0], reverse=True)

#데이타의 칼럼 수와 샘플 수 확인
first_year= True
for year, file in files_with_year:
    try:
        encoding ="cp949"
        df = pd.read_csv(file, encoding=encoding, nrows=0)
    except UnicodeDecodeError:
        encoding="utf-8"
        df= pd.read_csv(file, encoding=encoding, nrows=0)
    with open(file, "r", encoding=encoding) as f:
        line_count=0
        for line in f:
            line_count +=1
        print(f"{year}년: {line_count:,}개의 샘플, {len(df.columns)}개의 칼럼.")
    if first_year :
        col_to_read=df.columns
        first_year = False


# 입력 파일 갯수만큼 병합
df_list=[]
num_of_combined=int(input("병합할 파일 갯수:"))
if num_of_combined <= 0 : 
    import sys
    sys.exit()

for i in range(num_of_combined):
    print(f"{files_with_year[i][1]}파일을 읽습니다.")
    df= pd.read_csv(files_with_year[i][1], encoding=encoding)
    df=df.reindex()
    df= df[1:] #0행에 칼럼 영문 코드가 있는 경우가 종종 있음.
    df_list.append(df)

df=pd.concat(df_list, ignore_index=True) 
del df_list   
print(f"총 샘플 수는 {len(df)}")


#불필요한 _LABEL 칼럼 제거
all_col = df.columns.tolist()
col_del_list=[col for col in all_col if re.search("_LABEL", col)]
df= df.drop(columns=col_del_list, axis=1)
print(f"{len(col_del_list)}개의 _LABEL 칼럼 삭제")

# 임금 값이 결측 또는 0인 샘플은 삭제
from pandas.api.types import is_integer_dtype
if is_integer_dtype(df['최근3개월간평균급여']):
    df= df[df['최근3개월간평균급여'] != 0]
    

#dType이 object인 열을 레이블인지 결측인지 판단
col_object= df.select_dtypes(include=['object']).columns.tolist()
col_code_keyword =['코드','여부']
col_code=[]
for col in col_object:
    if any(keyword in col for keyword in col_code_keyword):
        col_code.append(col)
col_error = [col for col in col_object if col not in col_code]
print(f"Object 칼럼은 {len(col_object)}개.\n그 중 '코드'와 '여부'를 포함하는 칼럼은 {len(col_code)}개.\n나머지 (데이타가 고르지 못한) 칼럼은 {len(col_error)}개로 아래와 같음.\n{col_error}")

for col in col_object:
    try:
        df[col]=df[col].astype(int)
    except Exception as e:
        print(f"{col}칼럼, {type(e)}")
a=len(col_object)
col_object= df.select_dtypes(include=['object']).columns.tolist()
b=len(col_object)
print(f"{a-b}개의 칼럼이 처리. 여전히 {b}개의 Object 존재.")
df[col_object]




# 결측이 33%넘는 Object 피쳐 제거
sample_num = len(df)
for col in col_object:
    count = (df[col] == '0').sum()
    if count/sample_num > 0.33 :
        print(f"Object에서 결측이 33%를 넘는 항목:{col}.")
        y = input("삭제?(y/n)")
        if y== 'y':
            df= df.drop(columns=col, axis=1)
 

# 결측이 33%를 넘는 INT/FLOAT 항목 삭제
zero_list=[]
all_col = df.columns.tolist()
for col in all_col:
    count = (df[col]== 0).sum()
    if count/sample_num > 0.33 :
        zero_list.append(col)
 
print(f"Int/Float에서 결측이 33%를 넘는 항목:총{len(df.columns)}개 칼럼 중에 {len(zero_list)}개.\n")
print(zero_list)

col_qualified = set(all_col) - set(zero_list)
print(f"\n\n결측이 33% 이하인 칼럼은 {len(col_qualified)}개\n {col_qualified}")

df=df.drop(zero_list, axis=1)
print(f"\n\n결측 33% 넘는 {len(zero_list)}개 항목 삭제 완료.")


# 결측이 33%를 넘는 INT/FLOAT 항목 삭제
zero_list=[]
all_col = df.columns.tolist()
for col in all_col:
    count = (df[col]== 0).sum()
    if count/sample_num > 0.33 :
        zero_list.append(col)
 
print(f"Int/Float에서 결측이 33%를 넘는 항목:총{len(df.columns)}개 칼럼 중에 {len(zero_list)}개.\n")
print(zero_list)

col_qualified = set(all_col) - set(zero_list)
print(f"\n\n결측이 33% 이하인 칼럼은 {len(col_qualified)}개\n {col_qualified}")

df=df.drop(zero_list, axis=1)
print(f"\n\n결측 33% 넘는 {len(zero_list)}개 항목 삭제 완료.")


#NaN 값 많은 칼럼 삭제
for col in df.columns.tolist():
   if df[col].isnull().sum()/len(df) >0.33 :
       df=df.drop(columns=col, axis=1)
       print(f"NaN이 33% 이상인 {col} 칼럼을 삭제.")

#문자 코드 범주 인코딩
df_dtypes=df.dtypes
indices = df_dtypes.index[df_dtypes == 'object']
code_list=[]
from sklearn.preprocessing import LabelEncoder
for col in indices:
    le=LabelEncoder()
    df[col]=le.fit_transform(df[col])
    code_list.append((col, dict(zip(le.classes_,le.transform(le.classes_)))))
    print(f"{col}의 코드를 숫자로 변환")

#기타 의미 없는 칼럼 삭제
del_list=['동부읍면부코드']
for col in del_list:
    df=df.drop(columns=col, axis=1)

#코드 정보 CSV 만들기
file_path_excel=input("항목정보가 있는 파일명(엑셀):")
code_info=pd.read_excel(os.path.join(folder, file_path_excel), sheet_name="코드정보", header=1,)
code_info= code_info[['항목명','코드','코드의미 및 단위']]
code_info['항목명']=code_info['항목명'].fillna(method='ffill')
code_col='항목명'
for i in range(len(code_list)):
    code_bool= code_info[code_col] == code_list[i][0]
    code_info.loc[code_bool, '코드']= code_info.loc[code_bool, '코드'].map(code_list[i][1])

#저장

code_info.to_csv(os.path.join(folder, "항목정보.csv"), encoding="cp949")
print("항목정보.csv 파일을 생성.")

saveornot= input(f"모든 처리가 종료. {len(df)}개 Data를 저장?(y/n)")

if saveornot=="y":
    now=datetime.datetime.now()
    now_fomatted=now.strftime("%m-%d_%H-%M")
    df_filename= f"경제활동인구근로형태별DB_{now_fomatted}.csv"
    print("파일 저장 중...............")
    df.to_csv(os.path.join(folder,df_filename), encoding="cp949")
    print(f"{df_filename}을 저장.")
    del df


데이터가 있는 폴더: data


2018_8월_근로형태별_20250826_42207.csv
2019_8월_근로형태별_20250825_87431.csv
2021_8월_근로형태별_20250827_41497.csv
2022_8월_근로형태별_20250827_41497.csv
2023_8월_근로형태별_20250826_22167.csv
target_data.csv
weight_data.csv
총7개의 csv 파일을 찾았습니다

2023년: 58,135개의 샘플, 124개의 칼럼.
2022년: 57,385개의 샘플, 124개의 칼럼.
2021년: 58,883개의 샘플, 126개의 칼럼.
2019년: 60,160개의 샘플, 126개의 칼럼.
2018년: 61,097개의 샘플, 125개의 칼럼.


병합할 파일 갯수: 5


data\2023_8월_근로형태별_20250826_22167.csv파일을 읽습니다.
data\2022_8월_근로형태별_20250827_41497.csv파일을 읽습니다.
data\2021_8월_근로형태별_20250827_41497.csv파일을 읽습니다.
data\2019_8월_근로형태별_20250825_87431.csv파일을 읽습니다.
data\2018_8월_근로형태별_20250826_42207.csv파일을 읽습니다.
총 샘플 수는 295650
0개의 _LABEL 칼럼 삭제
Object 칼럼은 2개.
그 중 '코드'와 '여부'를 포함하는 칼럼은 2개.
나머지 (데이타가 고르지 못한) 칼럼은 0개로 아래와 같음.
[]
현재일관련사항_10차산업대분류코드칼럼, <class 'ValueError'>
1개의 칼럼이 처리. 여전히 1개의 Object 존재.
Int/Float에서 결측이 33%를 넘는 항목:총129개 칼럼 중에 76개.

['졸업연도', '현재일관련사항_지난주무급가족일근로여부', '현재일관련사항_지난주일시휴직여부', '현재일관련사항_지난주일시휴직사유코드', '현재일관련사항_평소1주간36시간미만근무사유코드', '현재일관련사항_지난주부업근무시간수', '현재일관련사항_지난주36시간미만근무사유코드', '현재일관련사항_추가취업전직희망코드', '현재일관련사항_추가취업가능성여부', '현재일관련사항_지난4주내추가취업구직여부', '현재일관련사항_고용계약기간코드', '구직사항_지난4주내구직여부', '구직사항_취업가능성유무', '구직사항_주요구직방법1코드', '구직사항_주요구직방법2코드', '구직사항_주요구직경로1코드', '구직사항_주요구직경로2코드', '구직사항_구직활동기간', '기타활동사항_취업희망여부', '기타활동사항_취업가능성유무', '기타활동사항_4주내비구직사유코드', '기타활동사항_지난1년내구직활동유무', '기타활동사항_지난주주요활동상태코드', '이전직장사항_전직유무', '이전직장사항_퇴직기간1년미만여부', '이전직장사항_1년미만퇴직연월', '이전직장사항_이직사유코드

항목정보가 있는 파일명(엑셀): 경제활동인구조사_8월_근로형태별(제공)_2023(코드집포함).xlsx


C:\Users\jwhan\AppData\Local\Temp\ipykernel_35760\1660556138.py:174: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  code_info['항목명']=code_info['항목명'].fillna(method='ffill')


항목정보.csv 파일을 생성.


모든 처리가 종료. 127679개 Data를 저장?(y/n) y


파일 저장 중...............
경제활동인구근로형태별DB_09-04_21-47.csv을 저장.
